# Paradise Papers

In 2016, confidential information mostly about offshore investments related to the legal firm Appleby were leaked to the Süddeutsche Zeitung, a german newspaper. After sharing the 13.4 million documents with the International Consortium of Investigative Journalists (ICIJ), the first results were published on the 5th of November 2017. It was the second big leak after the Panama Papers two years before, showing the offshore activities of some global companies as well as politicians such as prime ministers.

# The Paradise Papers Database

The Süddeutsche Zeitung and ICIJ will not publish all leaked documents to protect their source. However, they published a Neo4j database containing relations between companies, shareholders and legal firms mentioned in the Paradise Papers. 

The database consists of nodes with specific labels and relations connecting them. Each node or relation can store further properties. There are five different types / labels of nodes:
* Officer: A shareholder, either person or company
* Entity: An offshore company
* Intermediary: A person or company which helps registering offshore companies
* Address: An address with relation to one of the nodes
* Other: Nodes that do not fit in one of these categories, e.g. used for grouping of nodes

A relationship always connects two of these nodes and carries a label describing its function (e.g. REGISTERED_AT describing the relation between a company and the address of its headquarter).

You can either [download and execute it locally](https://offshoreleaks.icij.org/pages/database) (might take some time) or use a [sandbox provided by Neo4j](https://neo4j.com/sandbox-v2/) (sandbox expires after 3 days, discarding all changes, but a new instance can be started afterwards).

To execute the commands in this notebook, you will also need the [py2neo package](https://py2neo.org/v4/index.html) which you can install via pip and which serves as a driver for the Neo4j database. If you wish to render graphs, you will need the vis.py file and the vis.js library in addition, based on the [neo4j-jupyter notebooks by Nicole White](https://github.com/nicolewhite/neo4j-jupyter). Simple table outputs can be done via py2neo and do not require additional libraries.

You will need the following information to establish a connection to the database:
* ip-address and bolt port of the computer the database is running on
* username and password for authentication

If you use a sandbox, the necessary information is displayed in the details tab after startup.

# The Case of Nike

One of the global players mentioned in the Paradise Papers is Nike, the well known sneakers and sports brand. Although the journalists could not find any illegal activities within the leaked data, they still show a morally disputable system of tax avoidance.

Using a system of companies located in the Netherlands and the Bermudas, Nike could reduce its effective tax rate from the normal 35% in 2005 to approximately 13.2% in 2017. The details are described in an [article by the Süddeutsche Zeitung](https://projekte.sueddeutsche.de/paradisepapers/politik/nike-and-its-system-of-tax-avoidance-e727797/) (available in German and English). A description of the shape of the graph can be found [here](https://offshoreleaks-data.icij.org/offshoreleaks/neo4j/guide/index.html).

## Follow the Money

In this notebook, we will investigate Nike's internal company structure and follow the money from a pair of sneakers bought in Germany up to the headquarter of Nike in the United States.

### Establish a Connection to the Database

In [1]:
from py2neo import Graph

# schema of uri: "bolt://ip:port"
# schema of auth: ("username", "password")
graph = Graph(profile="bolt://some_ip:some_port", auth=("neo4j", "some_pwd"))

# import a function used to draw graphs in later sections
from vis.vis import drawSubgraph

# specify which property to display as label in a node
options = {
    "Officer": "name",
    "Entity": "name",
    "Intermediary": "name",
    "Address": "address"
}

physics = True

### Find Nike Retail & Nike Innovate

The article of Süddeutsche Zeitung mentions that most shoes bought in Europe are not sold by local companies (one for each country), but by central companies located in the Netherlands. The article mentions two names which we will use as starting points: Nike Retail BV and Nike Innovate CV.

First check if any company with this name exists in the database. Use toLower to transform the name to lower case and perform a case insensitive search.

In [2]:
query = """
MATCH (n)
WHERE toLower(n.name) CONTAINS 'nike retail' 
    OR toLower(n.name) CONTAINS 'nike innovate'
RETURN n.name
"""

data = graph.run(query).to_table()
if len(data) == 0:
    print("No record found")
data

No record found


### Find Companies Registered at Colosseum 1

Via the [Global Legal Entity Identifier Foundation](https://www.gleif.org/en/) we can lookup the address of the headquarter of any legal entity. Both Nike companies are registered at Colosseum 1 in Hilversum, Netherlands.

Find all nodes related to this address via a path p and display it as graph.

In [3]:
query = """
MATCH p = (a:Address) -- (n)
WHERE a.address = 'Colosseum 1'
RETURN p
"""

data = graph.run(query).to_subgraph()
drawSubgraph(data, options, height="350", filename="nike_colosseum1_direct_relation.html", physics=physics)

### Search for Nodes Related to Colosseum 1

Search for all nodes which are related to the address Colosseum 1 via a path p with a maximum length of three.

In [4]:
query = """
MATCH p = (a:Address) - [*0..3] - (n)
WHERE a.address = 'Colosseum 1'
RETURN p
"""

data = graph.run(query).to_subgraph()
drawSubgraph(data, options, height="800", filename="nike_colosseum1_path_length_3.html", physics=physics)

### Find All Nodes with Nike in Name

Find all nodes whose names contain the term Nike.

In [5]:
query = """
MATCH (n)
WHERE toLower(n.name) CONTAINS 'nike'
RETURN n.name, LABELS(n) as labels
"""

data = graph.run(query).to_table()
if len(data) == 0:
    print("No record found")
data

n.name,labels
"VS&B Niketan, Y-50, new #8, 9th Street, Annanagar; 600040 Chennai; India",['Address']
NIKE Jump Ltd.,['Entity']
NIKE INTERNATIONAL LTD.,['Entity']
Nike Finance Ltd.,['Entity']
NIKE Force,['Entity']
NIKE Pegasus,['Entity']
NIKE Flight,['Entity']
NIKE Lavadome,['Entity']
NIKE Waffle,['Entity']
NIKE Tailwind,['Entity']


### Nike Entities and their Jurisdiction

Count the number of entities with Nike in their name and group by jurisdiction.

In [6]:
query = """
MATCH (e:Entity)
WHERE toLower(e.name) CONTAINS 'nike'
RETURN e.jurisdiction_description AS jurisdiction, COUNT(*) AS numEntities
"""

data = graph.run(query).to_table()
if len(data) == 0:
    print("No record found")
data

jurisdiction,numEntities
Bermuda,11


### Find the Shortest Path Between Colosseum 1 and Nike Inc.

Find the shortest path p connecting the address Colosseum 1 with Nike's main company named 'NIKE, Inc.'.

In [7]:
query = """
MATCH p = SHORTESTPATH((a:Address) - [*] - (o:Officer))
WHERE a.address = 'Colosseum 1' 
    AND o.name = 'NIKE, Inc.'
RETURN p
"""

data = graph.run(query).to_subgraph()
drawSubgraph(data, options, height="400", filename="nike_shortest_path_colosseum1_nike.html", physics=physics)

### More Paths Between Colosseum 1 and Nike Inc.

Show all paths p between the address Colosseum 1 and Nike Inc. with a maximum length of five and without any nodes representing adresses besides the first node Colosseum 1.

In [8]:
query = """
MATCH p = (a:Address) - [*0..5] - (o:Officer)
WHERE a.address = 'Colosseum 1' 
    AND o.name = 'NIKE, Inc.' 
    AND NOT ANY(n IN NODES(p)[1..] WHERE n:Address)
RETURN p
"""

data = graph.run(query).to_subgraph()
drawSubgraph(data, options, height="800", filename="nike_paths_between colosseum1_nike.html", physics=physics)

### Only Paths with Officer Names Containing Nike

Restrict the previous example further to only contain paths where nodes of type Officer must have a name starting with Nike (essentially eliminating persons from the previous result) and the path contains the node Nike International Ltd.

In [9]:
query = """
MATCH p = (a:Address) - [*0..5] - (o:Officer)
WHERE a.address = 'Colosseum 1' 
    AND o.name = 'NIKE, Inc.' 
    AND NOT ANY(n IN NODES(p)[1..] WHERE n:Address)
    AND NOT ANY(n IN NODES(p) WHERE (n:Officer AND NOT toLower(n.name) STARTS WITH 'nike'))
    AND ANY(n IN NODES(p) WHERE n.name = 'NIKE INTERNATIONAL LTD.')
RETURN p
"""

data = graph.run(query).to_subgraph()
drawSubgraph(data, options, height="800", filename="nike_paths_between_colosseum1_nike_officers_nike.html", physics=physics)

### Connect Nike Officers and Entities with the Same Name

Connect all officers and entities with the same name starting with the term Nike with a 'SAME_NAME_AS' relationship. Use MERGE instead of CREATE to avoid duplicate relationships. Return the names of all such pairs found.

In [10]:
query = """
MATCH (o:Officer), (e:Entity)
WHERE toLower(o.name) STARTS WITH 'nike'
    AND o.name = e.name
MERGE (o) - [r:SAME_NAME_AS] -> (e)
RETURN o.name, e.name
"""

data = graph.run(query).to_table()
if len(data) == 0:
    print("No record found")
data

o.name,e.name
NIKE Jump Ltd.,NIKE Jump Ltd.
NIKE INTERNATIONAL LTD.,NIKE INTERNATIONAL LTD.
Nike Finance Ltd.,Nike Finance Ltd.
NIKE Pegasus,NIKE Pegasus
NIKE Flight,NIKE Flight
NIKE Lavadome,NIKE Lavadome
NIKE Waffle,NIKE Waffle
NIKE Tailwind,NIKE Tailwind
NIKE Cortez,NIKE Cortez
NIKE Huarache,NIKE Huarache


### Find Nike's Management Board

Find all persons which are directly related to a Nike company and the address of the headquarter at One Bowerman Drive. Order the results by names.

In [11]:
query = """
MATCH (n) -- (o:Officer) -- (a:Address)
WHERE toLower(n.name) STARTS WITH 'nike'
    AND a.address = 'One Bowerman Drive'
    AND NOT toLower(o.name) STARTS WITH 'nike'
RETURN DISTINCT o.name
ORDER BY o.name
"""

data = graph.run(query).to_table()
if len(data) == 0:
    print("No record found")
data

o.name
Blair - Donald Warren
Carter - James Cecil
Coburn III - John F
Denson - Charles
Hall - Kelly K
Knight - Philip H.
Krane - Hilary K
Miller - Ann M.
Peddie - Anthony
Pliska - Bernard F.


A quick web search reveals that most of these people had a high position at Nike, for example:
* Philip H. Knight: founder and former CEO
* Marcia Stilwell: former Vice President and Corporate Treasurer
* Lindsay D. Stewart: former Vice President and Chief of Staff
* Kelly K. Hall: former Vice President and CFO

### Explore the Complete Nike Universe

Detect all nodes which are directly related to a node with Nike in the beginning of its name.
In a first query, count the number of distinct relationships per type and in a second query show the resulting graph.

In [12]:
query1 = """
MATCH (n1) - [r] - (n2)
WHERE toLower(n1.name) STARTS WITH 'nike' 
RETURN TYPE(r) AS type, COUNT(DISTINCT r) AS num
ORDER BY num DESC
"""

data = graph.run(query1).to_table()
if len(data) == 0:
    print("No record found")
data

type,num
OFFICER_OF,113
REGISTERED_ADDRESS,37
CONNECTED_TO,12
SAME_NAME_AS,10


In [13]:
query2 = """
MATCH p = (n1) -- (n2)
WHERE toLower(n1.name) STARTS WITH 'nike'
RETURN p
"""

data = graph.run(query2).to_subgraph()
drawSubgraph(data, options, height="800", filename="nike_universe_directly_related.html", physics=physics)